In [9]:
import json
from tqdm import tqdm
import re
import torch
import pandas as pd
# 定义 color_mapping
color_mapping = {
    'red': [0.1, 0.15],
    'green': [0.15, 0.3],
    'blue': [0.3, 0.45],
    'yellow': [0.45, 0.6],
    'orange': [0.6, 0.75],
    'purple': [0.75, 0.9]
}

# 提取所有颜色
colors = color_mapping.keys()

# 检查颜色是否存在于字符串中
def find_colors_in_string(input_string):
    found_colors = [color for color in colors if color in input_string]
    return found_colors


def map_to_color(pixel):
    if pixel<0.1:
        return 'black'
    elif 0.1<=pixel<0.15:
        return 'red'
    elif 0.15<=pixel<0.3:
        return 'green'
    elif 0.3<=pixel<0.45:
        return 'blue'
    elif 0.45<=pixel<0.6:
        return 'yellow'
    elif 0.6<=pixel<0.75:
        return 'orange'
    elif 0.75<=pixel<=0.9:
        return 'purple'
    else:
        return 'other'

def compare_img(gen_img,gt_img):
    correct_pixel=0
    incorrect_pixel=0
    for i in range(len(gen_img)):
        for j in range(len(gen_img[i])):
            if map_to_color(gen_img[i][j])!=map_to_color(gt_img[i][j]):
                incorrect_pixel+=1
            else:
                correct_pixel+=1
    return correct_pixel,incorrect_pixel

acc=[]
answer_list=[i*45 for i in range(5,15)]
for answer in answer_list:
    #data_path=f'/datadrive_a/jihai/LLaVA/answer/answer-llava-v1.5-7b-mix-u-odd-{i}.jsonl'
    data_path=f'./answer/answer-llava-v1.5-7b-vq-vq_68ku_180km-sw-lora-{answer}.jsonl'
    time_count=0
    time_score=0
    weather_count=0
    weather_score=0
    position_count=0
    position_score=0
    battery_count=0
    battery_score=0
    with open (data_path, "r") as f:
        for line in tqdm(f):
            json_obj = json.loads(line.strip())
            ground_truth=json_obj['groun_truth']
            answer=json_obj['answer']
            prompt=json_obj['prompt']
            if prompt[0]!='<':
                continue
            if ' ' not in ground_truth:
                if ':' in ground_truth:
                    time_count+=1
                    pattern = r"(\d{2}):(\d{2}):(\d{2})"
                    match = re.search(pattern, ground_truth)
                    gt_h = int(match.group(1))
                    gt_m = int(match.group(2))
                    gt_s = int(match.group(3))
                    match = re.search(pattern, answer)
                    if match:
                        ans_h = int(match.group(1))
                        ans_m = int(match.group(2))
                        ans_s = int(match.group(3))
                        err_h=abs(ans_h - gt_h)
                        err_h=min(err_h,12-err_h)
                        err_m=abs(ans_m - gt_m)
                        err_m=min(err_m,60-err_m)
                        err_s=abs(ans_s - gt_s)
                        err_s=min(err_s,60-err_s)
                        err=(err_h/6.0 + err_m/30.0 + err_s/30.0)/3.0
    
                        time_score+=1-err
                elif 'sunny' in ground_truth or 'raining' in ground_truth or 'cloudy' in ground_truth:
                    weather_count+=1
                    if ground_truth in answer:
                        weather_score+=1
                elif '-' in ground_truth:
                    position_count+=1
                    if ground_truth in answer:
                        position_score+=1
                elif '%' in ground_truth:
                    battery_count+=1
                    match = re.search(r'\b(100|[1-9]\d?|0)%', ground_truth)
                    gt=int(match.group(1)) / 100
                    match = re.search(r'\b(100|[1-9]\d?|0)%', answer)
                    if match:
                        ans=int(match.group(1)) / 100
                        err=abs(ans - gt)
                        battery_score+=1-err
                else:
                    raise ValueError(f"Unknown ground truth format: {ground_truth}")
    time_acc=time_score/time_count
    weather_acc=weather_score/weather_count
    position_acc=position_score/position_count
    battery_acc=battery_score/battery_count
    total_acc=(time_score+weather_score+position_score+battery_score)/(time_count+weather_count+position_count+battery_count)
    acc.append([time_acc,weather_acc,position_acc,battery_acc,total_acc])
    print(f"time_acc: {time_acc:.4f}, weather_acc: {weather_acc:.4f}, position_acc: {position_acc:.4f}, battery_acc: {battery_acc:.4f}, total_acc: {total_acc:.4f}")
acc=torch.Tensor(acc)
acc=acc.permute(1,0)
print(f"time_acc:{acc[0]}")
print(f"weather_acc:{acc[1]}")
print(f"position_acc:{acc[2]}")
print(f"battery_acc:{acc[3]}")
print(f"total_acc:{acc[4]}")

columns = [f"{i*10}%" for i in range(1, 1+acc.shape[1])]
row_index = ['time_acc', 'weather_acc', 'position_acc', 'battery_acc', 'total_acc']
df = pd.DataFrame(data=acc.numpy(),
                  index=row_index,
                  columns=columns)
df.to_csv("output.csv", index=True, header=True)



499it [00:00, 163359.17it/s]


time_acc: 0.4787, weather_acc: 1.0000, position_acc: 1.0000, battery_acc: 0.8388, total_acc: 0.8521


499it [00:00, 178458.19it/s]


time_acc: 0.5025, weather_acc: 1.0000, position_acc: 1.0000, battery_acc: 0.9317, total_acc: 0.8794


499it [00:00, 174587.73it/s]


...

time_acc: 0.5400, weather_acc: 1.0000, position_acc: 1.0000, battery_acc: 0.9783, total_acc: 0.8984


499it [00:00, 169538.90it/s]


time_acc: 0.5375, weather_acc: 1.0000, position_acc: 1.0000, battery_acc: 0.9742, total_acc: 0.8969


499it [00:00, 157211.57it/s]

time_acc: 0.5600, weather_acc: 1.0000, position_acc: 1.0000, battery_acc: 0.9748, total_acc: 0.9017
time_acc:tensor([0.4787, 0.5025, 0.4879, 0.5117, 0.5562, 0.5213, 0.5175, 0.5400, 0.5375,
        0.5600])
weather_acc:tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
position_acc:tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
battery_acc:tensor([0.8388, 0.9317, 0.9647, 0.9617, 0.9668, 0.9690, 0.9745, 0.9783, 0.9743,
        0.9747])
total_acc:tensor([0.8521, 0.8794, 0.8842, 0.8885, 0.8990, 0.8922, 0.8928, 0.8984, 0.8969,
        0.9017])


In [ ]:
import json
data_path='/public_data/jihai/data/multimodalout/smart_watch_train_120ku_180km.json'
data_list=json.load(open(data_path, "r"))
print(len(data_list))

In [8]:
import matplotlib.pyplot as plt
import random
import os
from PIL import Image, ImageDraw, ImageFont,ImageColor

# sample=torch.load('/public_data/jihai/understanding/scripts/v1_5/answer/answer-llava-v1.5-7b-siglip-vq-sw-lora-450-image/11.pt')
# print(sample.shape)
# plt.imshow(sample)

gt_image_folder='/public_data/jihai/data/multimodalout/smart_watch_image_test'

answer_list=[i*45 for i in range(1,18)]
answer=answer_list[-1]
#data_path=f'/datadrive_a/jihai/LLaVA/answer/answer-llava-v1.5-7b-mix-u-odd-{i}.jsonl'
data_path=f'./answer/answer-llava-v1.5-7b-vq-vq_120ku_180km-2-sw-lora-{answer}.jsonl'
count=0
time_count=0
weather_count=0
position_count=0
battery_count=0
with open (data_path, "r") as f:
    for line in tqdm(f):
        json_obj = json.loads(line.strip())
        ground_truth=json_obj['groun_truth']
        answer=json_obj['answer']
        prompt=json_obj['prompt']
        if prompt[0]=='<':
            continue
        if ':' in prompt:
            time_count+=1
            # pattern = r"(\d{2}):(\d{2}):(\d{2})"
            # match = re.search(pattern, prompt)
            # gt_h = int(match.group(1))
            # gt_m = int(match.group(2))
            # gt_s = int(match.group(3))
           
        if 'sunny' in prompt or 'raining' in prompt or 'cloudy' in prompt:
            weather_count+=1
            
        if '-' in prompt:
            position_count+=prompt.count('-')
           
        if '%' in prompt:
            battery_count+=1
            match = re.search(r'\b(100|[1-9]\d?|0)%', prompt)
            gt=int(match.group(1)) / 100
            
        gt_image_path=os.path.join(gt_image_folder,json_obj['groun_truth_img_tensor'])
        gt_image=Image.open(gt_image_path)
        gen_image=torch.load(json_obj['output_img_file'])
        
        # plt.title(prompt)
        # plt.imshow(gt_image)
        # plt.imshow(gen_image)
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # figsize 可根据需要调整

        # 设置整个图的标题
        # fig.suptitle(prompt, fontsize=16)  # 总标题
        print(prompt)


        # 左边子图：显示 gt_image
        axes[0].imshow(gt_image)
        axes[0].set_title("Ground Truth Image")  # 子图标题
        axes[0].axis('off')  # 关闭坐标轴

        # 右边子图：显示 gen_image
        axes[1].imshow(gen_image)
        axes[1].set_title("Generated Image")  # 子图标题
        axes[1].axis('off')  # 关闭坐标轴

        # 调整布局以避免重叠
        plt.tight_layout(rect=[0, 0, 1, 0.9])  # 为总标题留出空间

        # 显示图像
        plt.savefig(f'./calculate/{count}.png')
        plt.close(fig)
        count+=1
        if count==30:
            break
print(f"time_count:{time_count}")
print(f"weather_count:{weather_count}")
print(f"position_count:{position_count}")
print(f"battery_count:{battery_count}")

2it [00:00, 11.61it/s]

Can you generate a image of a smart watch, displaying 02:07:00, showing raining weather.
Please show a smart watch, on which the current time is 02:49:02, with weather shown in the bottom-right corner.


8it [00:00, 15.56it/s]

Please show a smart watch, on which the time reads 07:19:04, with 39% battery.
Please show a smart watch, on which the current time is 11:09:02, with sunny weather displayed, with battery indicator at top-left.


14it [00:01, 13.10it/s]

...

Please show a smart watch, on which the current time is 00:57:03.
Please show a smart watch, showing the time 05:21:27, indicating cloudy conditions.


86it [00:03, 25.59it/s]

Please show a smart watch, displaying 04:51:26, displaying 78% power level.
Can you draw a smart watch, on which the time reads 09:34:06, showing cloudy weather.


88it [00:04, 21.75it/s]

time_count:30
weather_count:15
position_count:22
battery_count:13


In [ ]:
import re
g="<s> 00:50:38</s>"
pattern = r"(\d{2}):(\d{2}):(\d{2})"
match = re.search(pattern, g)
print(match.group(1))

In [18]:
12*131*3*45-180000

32220

In [12]:
1/3

0.3333333333333333

In [13]:
1/(3+1)

0.25

In [14]:
1/(3+2)

0.2

In [15]:
2/(3+1)

0.5

In [16]:
1.5/(3+0.5)

0.42857142857142855